In [1]:
D.<x> = PolynomialRing(QQ)
R.<x> = PolynomialRing(QQ)
S = PowerSeriesRing(QQ,'x', default_prec=1000)
L=LaurentSeriesRing(QQ,'x', default_prec=1000)
#S.<x,y>=PowerSeriesRing(QQ,default_prec=1000)
# Using "R = PolynomialRing(QQ,'x')" produces an error.
# Yet using S as defined here is fine (see definition of Fraleigh4 below)
# But S = PowerSeriesRing(QQ,'x','y') also produces an error.

def listmap(f,lst):return list(map(f,lst))

def uniteList(lst):
    ln=len(lst)
    if ln<2:return lst
    answer=[]
    for j in [0..ln-1]:
        lj=lst[j]
        if (lj in answer)==False:
            answer=answer+[lj]
    return answer

def delete_element(elt,ls):
    new=[]
    for j in ls:
        if j!=elt:new=new+[j]
    return new

def sortAndSmash(lst): #sorts and deletes repeats
    cropped=lst
    new=[]
    while len(cropped)>0:
        m=min(cropped)
        new=new+[m]
        cropped=delete_element(m,cropped)
    return(new)

def difference_table(lst):
    diffs=[]
    for k in [0..len(lst)-2]:
        diff=lst[k+1]-lst[k]
        diffs=diffs+[diff]
    return diffs

def evaluate(expression,incomingvalue):
    expr=str(expression)
    output=sage_eval(expr,locals={'x':incomingvalue})
    return output

P=Primes()

def prime(n): return P.unrank(n-1)

def unite(A,B):
    return set(A).union(B)

def factor_integer(n): #this is like "factor(integer)", but
    #the bases and exponents can be easily extracted.
    F=factor(n)
    return list(F)

def orderInteger(p,n):
    if n!=0:
        lst=n.digits(p)
        count=0
        k=0
        while lst[k]==0:
            count=count+1
            k=k+1
        return count
    if n==0:return(Infinity)

def order(p,fr):
    nn=numerator(QQ(fr))
    dn=denominator(QQ(fr))
    return orderInteger(p,nn)-orderInteger(p,dn)

def digitsum(p,n):return add(n.digits(p))

def digitsum(p,n):
    return sum(n.digits(p))

def rmnjntau(n):
    answer=0
    for k in [1..n-1]:
        answer=answer+(6*k-4*(n-k))*sigma(k,3)*sigma(n-k,5)
    answer=(5*sigma(n,3)+7*sigma(n,5))*n/12-35*answer
    return answer

def pochhammer(x,n):
    return product ([x+k for k in [0..n-1]])

def cRaleigh2(alpha,beta,nu):
    pchhmrA = pochhammer(alpha,nu)
    pchhmrB = pochhammer(beta,nu)
    fnu=factorial(nu)
    answer = D(pchhmrA*pchhmrB/fnu^2)
    return answer

def term(alpha,beta,p):
    return D(1/(alpha + p) + 1/(beta + p) - 2/(1 + p))

def eRaleigh(alpha, beta, nu):
    return D(sum([term(alpha, beta, p) for p in [0..nu - 1]]))

def FstarRaleigh2(alpha,beta,u,terms):
    fsr = sum([cRaleigh2(alpha, beta, nu)*eRaleigh(alpha,beta,nu)*u^nu for nu in [1..terms]])
    return D(fsr)

def Fraleigh2(alpha,beta,u,terms): 
    return D(sum([cRaleigh2(alpha,beta,nu)*u^nu for nu in [0..terms]]))

def FstarRaleigh3(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fsr2 = FstarRaleigh2(alpha,beta,x,n)
    return D(fsr2)

def Fraleigh4(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fr2 = Fraleigh2(alpha,beta,x,n)
    return D(fr2)

def exNo3c(n,m):
    a1 = S(x)*exp(S(FstarRaleigh3(n,m,x)/Fraleigh4(n,m,x))).O(2*n)
    # 
    # S(x) instead of x in df of a1 to avoid an error message
    # after use of the stripQuotationMarks command below.
    a2 = S(a1).O(n+1)
    return a2 # applying S to argument of exp is key.

def J(n,m):
    exn = exNo3c(n+2,m)
    jnm = 1/exn.reverse()
    return jnm.O(n+1) # Jtest is faster than J, and they agree.
def normalJ(n,m):
    f = J(n,m)
    return f.truncate(n+1)

def Jcoefficient(n,m):
    f = J(n+2,m)
    f1 = f.truncate(n+1)
    f1=f1.list()
    f1=Set(f1)
    g1 = f.truncate(n)
    g1 = g1.list()
    g1 = set(g1)
    h = f1.difference(g1)
    h = h.list()
    return h[0]

def xjNew(n,m):
    def xjNewStep(n,m):
        f=J(n,m)
        fl=f.list()
        rt=0
        for i in [0..len(fl)-1]:
            rt=rt+fl[i]*x**i*(m**3*2**6)**i
        return rt
    return xjNewStep(n-1,m)

def j(n,m):return L(xjNew(n,m)/x)

def xjLeoNew(n,m,multiplier):
    def xjNewStep(n,m):
        f=J(n,m)
        fl=f.list()
        rt=0
        for i in [0..len(fl)-1]:
            rt=rt+fl[i]*x**i*(multiplier)**i
        return rt
    return xjNewStep(n-1,m)

def jLeo(n,m,multiplier):return L(xjLeoNew(n,m,multiplier)/x)

def JpowerConstantTerm(m,power):
    trun=J(power,m)
    trun=expand(trun^power)
    polyversion=expand(trun*x^power)
    constant=polynomialCoefficient(power,polyversion)
    return constant

def jpowerConstantTerm(m,power):
    trun=j(power,m)
    trun=expand(trun^power)
    polyversion=expand(trun*x^power)
    constant=polynomialCoefficient(power,polyversion)
    return constant

def polynomialCoefficient(n,poly):
    return poly.list()[n]

def leadingCoefficient(polynomial):
    dg=polynomial.degree()
    
    return polynomialCoefficient(dg,polynomial)

def factorpairToFactor(factorpair):
    (monomial,exponent)=factorpair
    return monomial^exponent

def numericalfactor(polynomial):
    fp=polynomial.factor()
    lst=[factorpairToFactor(factorpair) for factorpair in fp]
    return polynomial/product(lst)

def firstfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.first())

def lastfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.last())

def reduced(polynomial):
    return numericalfactor(polynomial)*lastfactor(polynomial)

def element(k,listo):
    if k>len(listo):return (print("bad k"))
    return listo[k-1]

def mobiusInverseOfFunction(g,n):
    dv=divisors(n)
    nd=len(dv)
    f=0
    for k in [1..nd]:
        dvk=element(k,dv)
        f=f+moebius(dvk)*g(n/dvk)
    return f

def mobiusInverseOfList(g,n):
    dv=divisors(n)
    nd=len(dv)
    f=0
    for k in [1..nd]:
        dvk=element(k,dv)
        gndvk=element(n/dvk,g)
        f=f+moebius(dvk)*gndvk
    return f

def drop_first_element(listo):
    lenth=len(listo)
    answer=[]
    for k in [1..lenth-1]:
        answer=answer+[listo[k]]
    return answer

def factorSeriesFromLists(lsst,bound):
    truncatedlist=drop_first_element(lsst)
    F=[element(1,truncatedlist)]
    nswr=[-mobiusInverseOfList(F,1)/1]
    for n in [2..bound]:
        summ=0
        for k in [1..n-1]:
            ekf=element(k,F)
            tnk=element(n-k,truncatedlist)
            summ=summ+ekf*tnk
        Fn=n*element(n,truncatedlist)-summ
        F=F+[Fn]
        xpn=-mobiusInverseOfList(F,n)/n
        nswr=nswr+[xpn]
    return nswr

def factorSeriesFromListsWithTracking(lsst,bound):
    truncatedlist=drop_first_element(lsst)
    F=[element(1,truncatedlist)]
    nswr=[-mobiusInverseOfList(F,1)/1]
    for n in [2..bound]:
        summ=0
        for k in [1..n-1]:
            ekf=element(k,F)
            tnk=element(n-k,truncatedlist)
            summ=summ+ekf*tnk
            print((n,k))
        Fn=n*element(n,truncatedlist)-summ
        F=F+[Fn]
        xpn=-mobiusInverseOfList(F,n)/n
        nswr=nswr+[xpn]
        print("answer length: ",len(nswr))
    return nswr

def EulerMultiplication(listo):
    # Lemma 2.11 from my 1998 Experimental Mathematics paper.
    # Here the set A in the lemma is {1, 2, ...} = Z^+.
    p_A=[1] # so p_A[0]=1 as on top of p. 266, column 2.
    def ex(n):return listo[n-1] # ex is the exponent on the factor 1-x^n: -f(n)/n.
    def f(n):return -n*ex(n)
    answer=[1]
    def f_A(k):
        summ=0
        dk=divisors(k)
        for d in dk: #Here is where A = Z^+ is used to drop a constraint on k.
            summ=summ+f(d)
        return summ
    for n in [1..len(listo)-1]:
        sm=0
        for k in [1..n]:
            sm=sm+f_A(k)*p_A[n-k]
        p_A=p_A+[sm/n]
    return p_A

def listToPoly(listo):
    lenth=len(listo)
    answer=0
    for k in [1..lenth]:
        cf=listo[k-1]
        answer=answer+cf*x^(k-1)
    return answer

def stripQuotationMarks(string):
    var('x')
    return sage_eval(string,locals={'x':x})

print("okay1")

okay1


In [8]:
for n in [1..5]:
    print((n,sigma(n,k=11)))

(1, 1)
(2, 2049)
(3, 177148)
(4, 4196353)
(5, 48828126)


In [2]:
def RamanujanCongruenceSeries(n): 
    ans=0
    for a in [1..n+1]:
        term=a*sigma(a,k=1) 
        ans=ans+term*x^a
    return D(ans)

def recip(n):
    ans=L(1/RamanujanCongruenceSeries(n))
    ans=ans.O(n+1)
    return ans

def recipPowerConstantTerm(k):
    g=recip(k)^k
    g=g.O(2*k)
    return g[0]
    

L=LaurentSeriesRing(QQ,'x', default_prec=2010)

t2=[]
t3=[]
for a in [1..1000]: 
    c=recipPowerConstantTerm(a)
    o2=order(2,c)
    o3=order(3,c)
    ds2=digitsum(2,a)
    ds3=digitsum(3,a)
    test2=3*ds2-o2
    test3=ds3-o3
    if (mod(a,2)==0)&(test2!=0):
        t2=t2+[a]
    if (mod(a,3)==0)&(test3!=0):
        t3=t3+[a]
    if mod(a,10)==0:print(a)
print();print(t2);print();print(t3)

10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990
1000

[2, 6, 10, 14, 18, 22, 26, 30, 34, 38, 42, 46, 50, 54, 58, 62, 66, 70, 74, 78, 82, 86, 90, 94, 98, 102, 106, 110, 114, 118, 122, 126, 130, 134, 138, 142, 146, 150, 154, 158, 162, 166, 170, 174, 178, 182, 186, 190, 194, 198, 202, 206, 210, 214, 218, 222, 226, 230, 234, 238, 242, 246, 250, 254, 258, 262, 266, 270, 274, 278, 282, 286, 290, 294, 298, 302, 306, 310, 314, 318, 322, 326, 330, 334, 338, 342, 346, 350, 354, 358, 362, 366, 370, 374, 378, 382, 386, 390, 394, 398, 402, 406, 410, 414, 418, 422, 426, 430, 434, 438, 442, 446, 450, 454, 458, 462, 466, 470, 474, 478, 482, 486, 490, 494, 498, 502, 506

In [9]:
def RamanujanCongruenceSeries(n): 
    ans=0
    for a in [1..n+1]:
        term=a*sigma(a,k=1) 
        ans=ans+term*x^a
    return D(ans)

def recip(n):
    ans=L(1/RamanujanCongruenceSeries(n))
    ans=ans.O(n+1)
    return ans

def recipPowerConstantTerm(k):
    g=recip(k)^k
    g=g.O(2*k)
    return g[0]
    

L=LaurentSeriesRing(QQ,'x', default_prec=2010)

t2=[]
t3=[]
for a in [1..1000]: 
    c=recipPowerConstantTerm(a)
    o2=order(2,c)
    o3=order(3,c)
    ds2=digitsum(2,a)
    ds3=digitsum(3,a)
    test2=3*ds2-o2
    test3=ds3-o3
    if (mod(a,4)==0)&(test2!=0):
        t2=t2+[a]
    if (mod(a,3)==0)&(test3!=0):
        t3=t3+[a]
    if mod(a,10)==0:print(a)
print();print(t2);print();print(t3)

10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990
1000

[]

[]


In [4]:
def RamanujanCongruenceSeries(n): 
    ans=0
    for a in [1..n+1]:
        term=a*sigma(a,k=1) 
        ans=ans+term*x^a
    return D(ans)

def recip(n):
    ans=L(1/RamanujanCongruenceSeries(n))
    ans=ans.O(n+1)
    return ans

def recipPowerConstantTerm(k):
    g=recip(k)^k
    g=g.O(2*k)
    return g[0]
    

L=LaurentSeriesRing(QQ,'x', default_prec=2010)

data=[]
for a in [1..1000]: 
    c=recipPowerConstantTerm(a)
    o2=order(2,c)
    o3=order(3,c)
    ds2=digitsum(2,a)
    ds3=digitsum(3,a)
    test2=3*ds2-o2
    test3=ds3-o3
    if (mod(a,4)!=0)&(mod(a,3)!=0)&(test3==0):
        data=data+[a]
    if mod(a,10)==0:print(a)
print();print(data);print()

10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990
1000

[1, 7, 14, 25, 41, 47, 65, 71, 79, 95, 101, 122, 125, 137, 143, 146, 173, 179, 191, 241, 257, 263, 290, 299, 314, 338, 365, 371, 386, 389, 407, 413, 434, 443, 458, 497, 503, 515, 569, 578, 605, 683, 701, 727, 743, 749, 770, 785, 794, 830, 851, 854, 857, 866, 875, 893, 902, 929, 953, 986]



In [5]:
def RamanujanCongruenceSeries(n): 
    ans=0
    for a in [1..n+1]:
        term=a*sigma(a,k=1) 
        ans=ans+term*x^a
    return D(ans)

def recip(n):
    ans=L(1/RamanujanCongruenceSeries(n))
    ans=ans.O(n+1)
    return ans

def recipPowerConstantTerm(k):
    g=recip(k)^k
    g=g.O(2*k)
    return g[0]
    

L=LaurentSeriesRing(QQ,'x', default_prec=2010)

data=[]
for a in [1..1000]: 
    c=recipPowerConstantTerm(a)
    o2=order(2,c)
    o3=order(3,c)
    ds2=digitsum(2,a)
    ds3=digitsum(3,a)
    test2=3*ds2-o2
    test3=ds3-o3
    if (mod(a,4)==2)&(test3==0):
        data=data+[a]
    if mod(a,10)==0:print(a)
print();print(data);print()

10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990
1000

[6, 14, 18, 30, 42, 54, 66, 78, 90, 102, 114, 122, 126, 138, 146, 150, 162, 174, 186, 198, 210, 222, 234, 246, 258, 270, 282, 290, 294, 306, 314, 318, 330, 338, 342, 354, 366, 378, 386, 390, 402, 414, 426, 434, 438, 450, 458, 462, 474, 486, 498, 510, 522, 534, 546, 558, 570, 578, 582, 594, 606, 618, 630, 642, 654, 666, 678, 690, 702, 714, 726, 738, 750, 762, 770, 774, 786, 794, 798, 810, 822, 830, 834, 846, 854, 858, 866, 870, 882, 894, 902, 906, 918, 930, 942, 954, 966, 978, 986, 990]



In [7]:
for a in data:
    adp=a.digits(4)
    print((a,mod(a,8)))

(6, 6)
(14, 6)
(18, 2)
(30, 6)
(42, 2)
(54, 6)
(66, 2)
(78, 6)
(90, 2)
(102, 6)
(114, 2)
(122, 2)
(126, 6)
(138, 2)
(146, 2)
(150, 6)
(162, 2)
(174, 6)
(186, 2)
(198, 6)
(210, 2)
(222, 6)
(234, 2)
(246, 6)
(258, 2)
(270, 6)
(282, 2)
(290, 2)
(294, 6)
(306, 2)
(314, 2)
(318, 6)
(330, 2)
(338, 2)
(342, 6)
(354, 2)
(366, 6)
(378, 2)
(386, 2)
(390, 6)
(402, 2)
(414, 6)
(426, 2)
(434, 2)
(438, 6)
(450, 2)
(458, 2)
(462, 6)
(474, 2)
(486, 6)
(498, 2)
(510, 6)
(522, 2)
(534, 6)
(546, 2)
(558, 6)
(570, 2)
(578, 2)
(582, 6)
(594, 2)
(606, 6)
(618, 2)
(630, 6)
(642, 2)
(654, 6)
(666, 2)
(678, 6)
(690, 2)
(702, 6)
(714, 2)
(726, 6)
(738, 2)
(750, 6)
(762, 2)
(770, 2)
(774, 6)
(786, 2)
(794, 2)
(798, 6)
(810, 2)
(822, 6)
(830, 6)
(834, 2)
(846, 6)
(854, 6)
(858, 2)
(866, 2)
(870, 6)
(882, 2)
(894, 6)
(902, 6)
(906, 2)
(918, 6)
(930, 2)
(942, 6)
(954, 2)
(966, 6)
(978, 2)
(986, 2)
(990, 6)


In [13]:
for a in data:
    print((a/2,digitsum(2,a),order(2,(a-ZZ(mod(a,8))))))

(3, 2, +Infinity)
(7, 3, 3)
(9, 2, 4)
(15, 4, 3)
(21, 3, 3)
(27, 4, 4)
(33, 2, 6)
(39, 4, 3)
(45, 4, 3)
(51, 4, 5)
(57, 4, 4)
(61, 5, 3)
(63, 6, 3)
(69, 3, 3)
(73, 3, 4)
(75, 4, 4)
(81, 3, 5)
(87, 5, 3)
(93, 5, 3)
(99, 4, 6)
(105, 4, 4)
(111, 6, 3)
(117, 5, 3)
(123, 6, 4)
(129, 2, 8)
(135, 4, 3)
(141, 4, 3)
(145, 3, 5)
(147, 4, 5)
(153, 4, 4)
(157, 5, 3)
(159, 6, 3)
(165, 4, 3)
(169, 4, 4)
(171, 5, 4)
(177, 4, 5)
(183, 6, 3)
(189, 6, 3)
(193, 3, 7)
(195, 4, 7)
(201, 4, 4)
(207, 6, 3)
(213, 5, 3)
(217, 5, 4)
(219, 6, 4)
(225, 4, 6)
(229, 5, 3)
(231, 6, 3)
(237, 6, 3)
(243, 6, 5)
(249, 6, 4)
(255, 8, 3)
(261, 3, 3)
(267, 4, 4)
(273, 3, 5)
(279, 5, 3)
(285, 5, 3)
(289, 3, 6)
(291, 4, 6)
(297, 4, 4)
(303, 6, 3)
(309, 5, 3)
(315, 6, 4)
(321, 3, 7)
(327, 5, 3)
(333, 5, 3)
(339, 5, 5)
(345, 5, 4)
(351, 7, 3)
(357, 5, 3)
(363, 6, 4)
(369, 5, 5)
(375, 7, 3)
(381, 7, 3)
(385, 3, 8)
(387, 4, 8)
(393, 4, 4)
(397, 5, 3)
(399, 6, 3)
(405, 5, 3)
(411, 6, 4)
(415, 7, 3)
(417, 4, 6)
(423, 6, 3)
(427, 6

In [14]:
def f(n):return n/2
print(listmap(f,data))

[3, 7, 9, 15, 21, 27, 33, 39, 45, 51, 57, 61, 63, 69, 73, 75, 81, 87, 93, 99, 105, 111, 117, 123, 129, 135, 141, 145, 147, 153, 157, 159, 165, 169, 171, 177, 183, 189, 193, 195, 201, 207, 213, 217, 219, 225, 229, 231, 237, 243, 249, 255, 261, 267, 273, 279, 285, 289, 291, 297, 303, 309, 315, 321, 327, 333, 339, 345, 351, 357, 363, 369, 375, 381, 385, 387, 393, 397, 399, 405, 411, 415, 417, 423, 427, 429, 433, 435, 441, 447, 451, 453, 459, 465, 471, 477, 483, 489, 493, 495]


In [16]:
lst=[]
for a in data:
    lst=lst+[mod(a,3)]
print(lst)


[0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 2, 0, 0, 2, 0, 0, 0, 0, 2, 0, 0, 0, 0, 2, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 2, 0, 0, 0, 2, 0, 0, 2, 0, 2, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 2, 0]


In [18]:
lst=[]
for a in data:
    print((a,a.digits(9)))

(6, [6])
(14, [5, 1])
(18, [0, 2])
(30, [3, 3])
(42, [6, 4])
(54, [0, 6])
(66, [3, 7])
(78, [6, 8])
(90, [0, 1, 1])
(102, [3, 2, 1])
(114, [6, 3, 1])
(122, [5, 4, 1])
(126, [0, 5, 1])
(138, [3, 6, 1])
(146, [2, 7, 1])
(150, [6, 7, 1])
(162, [0, 0, 2])
(174, [3, 1, 2])
(186, [6, 2, 2])
(198, [0, 4, 2])
(210, [3, 5, 2])
(222, [6, 6, 2])
(234, [0, 8, 2])
(246, [3, 0, 3])
(258, [6, 1, 3])
(270, [0, 3, 3])
(282, [3, 4, 3])
(290, [2, 5, 3])
(294, [6, 5, 3])
(306, [0, 7, 3])
(314, [8, 7, 3])
(318, [3, 8, 3])
(330, [6, 0, 4])
(338, [5, 1, 4])
(342, [0, 2, 4])
(354, [3, 3, 4])
(366, [6, 4, 4])
(378, [0, 6, 4])
(386, [8, 6, 4])
(390, [3, 7, 4])
(402, [6, 8, 4])
(414, [0, 1, 5])
(426, [3, 2, 5])
(434, [2, 3, 5])
(438, [6, 3, 5])
(450, [0, 5, 5])
(458, [8, 5, 5])
(462, [3, 6, 5])
(474, [6, 7, 5])
(486, [0, 0, 6])
(498, [3, 1, 6])
(510, [6, 2, 6])
(522, [0, 4, 6])
(534, [3, 5, 6])
(546, [6, 6, 6])
(558, [0, 8, 6])
(570, [3, 0, 7])
(578, [2, 1, 7])
(582, [6, 1, 7])
(594, [0, 3, 7])
(606, [3, 4, 7])


In [24]:
def RamanujanCongruenceSeries(n): 
    ans=0
    for a in [1..n+1]:
        term=a*sigma(a,k=1) 
        ans=ans+term*x^a
    return D(ans)

def recip(n):
    ans=L(1/RamanujanCongruenceSeries(n))
    ans=ans.O(n+1)
    return ans

def recipPowerConstantTerm(k):
    g=recip(k)^k
    g=g.O(2*k)
    return g[0]
    

L=LaurentSeriesRing(QQ,'x', default_prec=2010)

data=[]
for a in [1..1000]: 
    if mod(a,3)==1:
        c=recipPowerConstantTerm(a)
        o2=order(2,c)
        o3=order(3,c)
        ds2=digitsum(2,a)
        ds3=digitsum(3,a)
        if o3==ds3:
            print((a,ds3,o3))
            data=data+[a]
print();print(data);print() # A164123		Partial sums of A162436.

(1, 1, 1)
(4, 2, 2)
(7, 3, 3)
(16, 4, 4)
(25, 5, 5)
(52, 6, 6)
(79, 7, 7)
(160, 8, 8)
(241, 9, 9)
(484, 10, 10)
(727, 11, 11)

[1, 4, 7, 16, 25, 52, 79, 160, 241, 484, 727]



In [26]:
for a in data:
    print(factor(a+2))

3
2 * 3
3^2
2 * 3^2
3^3
2 * 3^3
3^4
2 * 3^4
3^5
2 * 3^5
3^6


In [29]:
# bigger list of A164123:
ls=[1, 4, 7, 16, 25, 52, 79, 160, 241, 484, 727, 1456, 2185, 4372, 6559, 13120, 19681, 39364, 59047, 118096, 177145, 354292, 531439, 1062880, 1594321, 3188644, 4782967, 9565936, 14348905, 28697812, 43046719, 86093440, 129140161, 258280324, 387420487, 774840976, 1162261465]
for n in ls:
    print((n,factor(n+2)))

(1, 3)
(4, 2 * 3)
(7, 3^2)
(16, 2 * 3^2)
(25, 3^3)
(52, 2 * 3^3)
(79, 3^4)
(160, 2 * 3^4)
(241, 3^5)
(484, 2 * 3^5)
(727, 3^6)
(1456, 2 * 3^6)
(2185, 3^7)
(4372, 2 * 3^7)
(6559, 3^8)
(13120, 2 * 3^8)
(19681, 3^9)
(39364, 2 * 3^9)
(59047, 3^10)
(118096, 2 * 3^10)
(177145, 3^11)
(354292, 2 * 3^11)
(531439, 3^12)
(1062880, 2 * 3^12)
(1594321, 3^13)
(3188644, 2 * 3^13)
(4782967, 3^14)
(9565936, 2 * 3^14)
(14348905, 3^15)
(28697812, 2 * 3^15)
(43046719, 3^16)
(86093440, 2 * 3^16)
(129140161, 3^17)
(258280324, 2 * 3^17)
(387420487, 3^18)
(774840976, 2 * 3^18)
(1162261465, 3^19)


In [30]:
def RamanujanCongruenceSeries(n): 
    ans=0
    for a in [1..n+1]:
        term=a*sigma(a,k=1) 
        ans=ans+term*x^a
    return D(ans)

def recip(n):
    ans=L(1/RamanujanCongruenceSeries(n))
    ans=ans.O(n+1)
    return ans

def recipPowerConstantTerm(k):
    g=recip(k)^k
    g=g.O(2*k)
    return g[0]
    

L=LaurentSeriesRing(QQ,'x', default_prec=2010)

data=[]
for a in [1..1000]: 
    if mod(a,3)==2:
        c=recipPowerConstantTerm(a)
        o2=order(2,c)
        o3=order(3,c)
        ds2=digitsum(2,a)
        ds3=digitsum(3,a)
        if o3==ds3:
            print((a,ds3,o3))
            data=data+[a]
print();print(data);print() 

(14, 4, 4)
(20, 4, 4)
(41, 5, 5)
(47, 5, 5)
(56, 4, 4)
(65, 5, 5)
(71, 7, 7)
(95, 5, 5)
(101, 5, 5)
(122, 6, 6)
(125, 7, 7)
(128, 6, 6)
(137, 5, 5)
(143, 7, 7)
(146, 6, 6)
(164, 4, 4)
(173, 5, 5)
(179, 7, 7)
(191, 5, 5)
(200, 6, 6)
(224, 8, 8)
(257, 5, 5)
(263, 5, 5)
(284, 6, 6)
(290, 6, 6)
(299, 5, 5)
(308, 6, 6)
(314, 8, 8)
(338, 6, 6)
(344, 6, 6)
(365, 7, 7)
(368, 8, 8)
(371, 7, 7)
(380, 6, 6)
(386, 8, 8)
(389, 7, 7)
(407, 5, 5)
(413, 7, 7)
(416, 6, 6)
(434, 6, 6)
(440, 8, 8)
(443, 7, 7)
(458, 10, 10)
(488, 4, 4)
(497, 5, 5)
(503, 7, 7)
(515, 5, 5)
(524, 6, 6)
(548, 8, 8)
(569, 5, 5)
(578, 6, 6)
(584, 8, 8)
(596, 6, 6)
(605, 7, 7)
(620, 10, 10)
(656, 8, 8)
(683, 9, 9)
(701, 11, 11)
(743, 5, 5)
(749, 5, 5)
(770, 6, 6)
(776, 6, 6)
(785, 5, 5)
(794, 6, 6)
(800, 8, 8)
(824, 6, 6)
(830, 6, 6)
(851, 7, 7)
(854, 8, 8)
(857, 7, 7)
(866, 6, 6)
(872, 8, 8)
(875, 7, 7)
(893, 5, 5)
(902, 6, 6)
(908, 8, 8)
(920, 6, 6)
(929, 7, 7)
(953, 9, 9)
(986, 6, 6)
(992, 6, 6)

[14, 20, 41, 47, 56, 65, 71, 

In [77]:
data=[14, 20, 41, 47, 56, 65, 71, 95, 101, 122, 125, 128, 137, 143, 146, 164, 173, 179, 191, 200, 224, 257, 263, 284, 290, 299, 308, 314, 338, 344, 365, 368, 371, 380, 386, 389, 407, 413, 416, 434, 440, 443, 458, 488, 497, 503, 515, 524, 548, 569, 578, 584, 596, 605, 620, 656, 683, 701, 743, 749, 770, 776, 785, 794, 800, 824, 830, 851, 854, 857, 866, 872, 875, 893, 902, 908, 920, 929, 953, 986, 992]
primes=[]
for a in data:
    b=ZZ((a+1)/3)
    print((a,factor(b)))
    if is_prime(b):
        primes=primes+[b]
print();print(primes);print()

(14, 5)
(20, 7)
(41, 2 * 7)
(47, 2^4)
(56, 19)
(65, 2 * 11)
(71, 2^3 * 3)
(95, 2^5)
(101, 2 * 17)
(122, 41)
(125, 2 * 3 * 7)
(128, 43)
(137, 2 * 23)
(143, 2^4 * 3)
(146, 7^2)
(164, 5 * 11)
(173, 2 * 29)
(179, 2^2 * 3 * 5)
(191, 2^6)
(200, 67)
(224, 3 * 5^2)
(257, 2 * 43)
(263, 2^3 * 11)
(284, 5 * 19)
(290, 97)
(299, 2^2 * 5^2)
(308, 103)
(314, 3 * 5 * 7)
(338, 113)
(344, 5 * 23)
(365, 2 * 61)
(368, 3 * 41)
(371, 2^2 * 31)
(380, 127)
(386, 3 * 43)
(389, 2 * 5 * 13)
(407, 2^3 * 17)
(413, 2 * 3 * 23)
(416, 139)
(434, 5 * 29)
(440, 3 * 7^2)
(443, 2^2 * 37)
(458, 3^2 * 17)
(488, 163)
(497, 2 * 83)
(503, 2^3 * 3 * 7)
(515, 2^2 * 43)
(524, 5^2 * 7)
(548, 3 * 61)
(569, 2 * 5 * 19)
(578, 193)
(584, 3 * 5 * 13)
(596, 199)
(605, 2 * 101)
(620, 3^2 * 23)
(656, 3 * 73)
(683, 2^2 * 3 * 19)
(701, 2 * 3^2 * 13)
(743, 2^3 * 31)
(749, 2 * 5^3)
(770, 257)
(776, 7 * 37)
(785, 2 * 131)
(794, 5 * 53)
(800, 3 * 89)
(824, 5^2 * 11)
(830, 277)
(851, 2^2 * 71)
(854, 3 * 5 * 19)
(857, 2 * 11 * 13)
(866, 17^2)
(8

In [80]:
data=[14, 20, 41, 47, 56, 65, 71, 95, 101, 122, 125, 128, 137, 143, 146, 164, 173, 179, 191, 200, 224, 257, 263, 284, 290, 299, 308, 314, 338, 344, 365, 368, 371, 380, 386, 389, 407, 413, 416, 434, 440, 443, 458, 488, 497, 503, 515, 524, 548, 569, 578, 584, 596, 605, 620, 656, 683, 701, 743, 749, 770, 776, 785, 794, 800, 824, 830, 851, 854, 857, 866, 872, 875, 893, 902, 908, 920, 929, 953, 986, 992]
primes=[]
for a in data:
    b=ZZ((a+1)/3)
    print((a,factor(b)))
    if is_prime(b):
        primes=primes+[(b,kronecker_symbol(b,3))]
print();print(primes);print()
test2=[]
test1=[]
for n in [1..331]:
    if (is_prime(n))&(mod(n,3)==2):test2=test2+[n]
    if (is_prime(n))&(mod(n,3)==1):test1=test1+[n]
print();print(test1);print();print(test2)

(14, 5)
(20, 7)
(41, 2 * 7)
(47, 2^4)
(56, 19)
(65, 2 * 11)
(71, 2^3 * 3)
(95, 2^5)
(101, 2 * 17)
(122, 41)
(125, 2 * 3 * 7)
(128, 43)
(137, 2 * 23)
(143, 2^4 * 3)
(146, 7^2)
(164, 5 * 11)
(173, 2 * 29)
(179, 2^2 * 3 * 5)
(191, 2^6)
(200, 67)
(224, 3 * 5^2)
(257, 2 * 43)
(263, 2^3 * 11)
(284, 5 * 19)
(290, 97)
(299, 2^2 * 5^2)
(308, 103)
(314, 3 * 5 * 7)
(338, 113)
(344, 5 * 23)
(365, 2 * 61)
(368, 3 * 41)
(371, 2^2 * 31)
(380, 127)
(386, 3 * 43)
(389, 2 * 5 * 13)
(407, 2^3 * 17)
(413, 2 * 3 * 23)
(416, 139)
(434, 5 * 29)
(440, 3 * 7^2)
(443, 2^2 * 37)
(458, 3^2 * 17)
(488, 163)
(497, 2 * 83)
(503, 2^3 * 3 * 7)
(515, 2^2 * 43)
(524, 5^2 * 7)
(548, 3 * 61)
(569, 2 * 5 * 19)
(578, 193)
(584, 3 * 5 * 13)
(596, 199)
(605, 2 * 101)
(620, 3^2 * 23)
(656, 3 * 73)
(683, 2^2 * 3 * 19)
(701, 2 * 3^2 * 13)
(743, 2^3 * 31)
(749, 2 * 5^3)
(770, 257)
(776, 7 * 37)
(785, 2 * 131)
(794, 5 * 53)
(800, 3 * 89)
(824, 5^2 * 11)
(830, 277)
(851, 2^2 * 71)
(854, 3 * 5 * 19)
(857, 2 * 11 * 13)
(866, 17^2)
(8

In [37]:
for a in data:
    print((a,factor((a+1)/3)))

(14, 5)
(20, 7)
(41, 2 * 7)
(47, 2^4)
(56, 19)
(65, 2 * 11)
(71, 2^3 * 3)
(95, 2^5)
(101, 2 * 17)
(122, 41)
(125, 2 * 3 * 7)
(128, 43)
(137, 2 * 23)
(143, 2^4 * 3)
(146, 7^2)
(164, 5 * 11)
(173, 2 * 29)
(179, 2^2 * 3 * 5)
(191, 2^6)
(200, 67)
(224, 3 * 5^2)
(257, 2 * 43)
(263, 2^3 * 11)
(284, 5 * 19)
(290, 97)
(299, 2^2 * 5^2)
(308, 103)
(314, 3 * 5 * 7)
(338, 113)
(344, 5 * 23)
(365, 2 * 61)
(368, 3 * 41)
(371, 2^2 * 31)
(380, 127)
(386, 3 * 43)
(389, 2 * 5 * 13)
(407, 2^3 * 17)
(413, 2 * 3 * 23)
(416, 139)
(434, 5 * 29)
(440, 3 * 7^2)
(443, 2^2 * 37)
(458, 3^2 * 17)
(488, 163)
(497, 2 * 83)
(503, 2^3 * 3 * 7)
(515, 2^2 * 43)
(524, 5^2 * 7)
(548, 3 * 61)
(569, 2 * 5 * 19)
(578, 193)
(584, 3 * 5 * 13)
(596, 199)
(605, 2 * 101)
(620, 3^2 * 23)
(656, 3 * 73)
(683, 2^2 * 3 * 19)
(701, 2 * 3^2 * 13)
(743, 2^3 * 31)
(749, 2 * 5^3)
(770, 257)
(776, 7 * 37)
(785, 2 * 131)
(794, 5 * 53)
(800, 3 * 89)
(824, 5^2 * 11)
(830, 277)
(851, 2^2 * 71)
(854, 3 * 5 * 19)
(857, 2 * 11 * 13)
(866, 17^2)
(8

In [39]:
kronecker_symbol?

In [57]:
for a in data:
    b=(a+1)/3
    if is_prime(ZZ(b))==True:
        ks=kronecker_symbol(2,b)
        print((a,b,ks))

(14, 5, -1)
(20, 7, 1)
(56, 19, -1)
(122, 41, 1)
(128, 43, -1)
(200, 67, -1)
(290, 97, 1)
(308, 103, 1)
(338, 113, 1)
(380, 127, 1)
(416, 139, -1)
(488, 163, -1)
(578, 193, 1)
(596, 199, 1)
(770, 257, 1)
(830, 277, -1)
(920, 307, -1)
(992, 331, -1)


In [58]:
def RamanujanCongruenceSeries(n): 
    ans=0
    for a in [1..n+1]:
        term=a*sigma(a,k=1) 
        ans=ans+term*x^a
    return D(ans)

def recip(n):
    ans=L(1/RamanujanCongruenceSeries(n))
    ans=ans.O(n+1)
    return ans

def recipPowerConstantTerm(k):
    g=recip(k)^k
    g=g.O(2*k)
    return g[0]
    

L=LaurentSeriesRing(QQ,'x', default_prec=3010)

data=[]
for a in [1..3000]: 
    if mod(a,3)==1:
        c=recipPowerConstantTerm(a)
        o2=order(2,c)
        o3=order(3,c)
        ds2=digitsum(2,a)
        ds3=digitsum(3,a)
        if o3==ds3:
            print((a,ds3,o3))
            data=data+[a]
print();print(data);print() # A164123		Partial sums of A162436.

(1, 1, 1)
(4, 2, 2)
(7, 3, 3)
(16, 4, 4)
(25, 5, 5)
(52, 6, 6)
(79, 7, 7)
(160, 8, 8)
(241, 9, 9)
(484, 10, 10)
(727, 11, 11)
(1456, 12, 12)
(2185, 13, 13)

[1, 4, 7, 16, 25, 52, 79, 160, 241, 484, 727, 1456, 2185]

